In [1]:
import pandas as pd
import calmap
import numpy as np
import os

In [2]:
from pathlib import Path
from os import environ, write

import re
from datetime import datetime, time
import datetime as dt
from collections import defaultdict

import requests

In [3]:
def load_data():
    if environ.get("USE_LOCAL_FILES"):
        paths = Path("extracted").rglob("*.csv")
        df_parts = []

        for path in sorted(paths):

            print("Reading for analysis:", path)

            with open(path, "r", encoding="utf-8") as fp:
                df_part = pd.read_csv(fp, delimiter=",", dtype=str)
                df_parts.append(df_part)

            # if df is None:
            #     df = df_part
            # else:
            #     df = df.append(df_part, ignore_index=True)

        df = pd.concat(df_parts, ignore_index=True)

    else:
        raise NotImplementedError("Remote data not yet implemented")
    
    df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors = 'coerce')
    df['date_of_proceeding'] = pd.to_datetime(df['date_of_proceeding'], errors = 'coerce')
    df['date_of_publication'] = pd.to_datetime(df['date_of_publication'], errors = 'coerce')

    return df

In [4]:
df = load_data()

Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-01-17T08-05-41.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-01-24T08-05-43.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-01-31T03-50-40.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-02-07T03-47-56.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-02-14T03-54-29.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-02-21T04-08-08.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-02-28T04-20-16.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-03-07T07-20-48.jsonl.csv
Reading for analysis: extracted\buckets\insolven

Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-10-20T00-26-46.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-10-20T07-31-07.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-10-20T13-43-00.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-10-20T20-17-09.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-10-21T06-38-42.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-10-21T13-46-12.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-10-21T19-37-11.jsonl.csv
Reading for analysis: extracted\buckets\insolvenzenard\insolvenzbekanntmachungen-scraper\2021-10-22T07-02-46.jsonl.csv
Reading for analysis: extracted\buckets\insolven

In [5]:
df.drop_duplicates(subset ="description_hash", keep = "last", inplace = True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404331 entries, 987478 to 1795161
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   _key                 0 non-null       object        
 1   case_nr              404331 non-null  object        
 2   court                404331 non-null  object        
 3   date_of_birth        283552 non-null  datetime64[ns]
 4   date_of_proceeding   356040 non-null  datetime64[ns]
 5   date_of_publication  404331 non-null  datetime64[ns]
 6   federal_state        404331 non-null  object        
 7   file_name            0 non-null       object        
 8   kind                 403994 non-null  object        
 9   title                0 non-null       object        
 10  type_of_proceeding   404331 non-null  object        
 11  zipcode              402414 non-null  object        
 12  detail_form_name     404331 non-null  object        
 13  format  

# Bundesweit

#### Replace old state names

In [7]:
replacements = {
    "nw": "Nordrhein-Westfalen",
    "bb": "Brandenburg",
    "sh": "Schleswig-Holstein",
    "ns": "Niedersachsen",
    "by": "Bayern",
    "rp": "Rheinland-Pfalz",
    "sn": "Sachsen",
    "bw": "Baden-Württemberg",
    "be": "Berlin",
    "st": "Brandenburg",
    "he": "Hessen",
    "hh": "Hamburg",
    "sl": "Saarland",
    "hb": "Bremen",
    "th": "Thüringen",
    "mv": "Mecklenburg-Vorpommern",
}

for old_name, new_name in replacements.items():
    df = df.replace(to_replace = [old_name], value=new_name)

### Filtern nach Insolvenzart

In [11]:
FILTER_ART = "in"

In [12]:
data = df[df['kind'] == FILTER_ART]

In [13]:
data_eröffnet = data[data['type_of_proceeding'].isin(
    ['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren', 'Eroeffnungen']
)]

In [166]:
# Filter nach Bundesland
#data_nw = data_eröffnet[data_eröffnet["federal_state"].isin(['Nordrhein-Westfalen'])]
data_nw = data_eröffnet

In [167]:
# Filter nach Zeitraum
date_range_nw = data_nw[data_nw.date_of_publication.between('2021-08-01', '2021-11-07')]

In [168]:
date_range_nw.sample(25)

,_key,case_nr,court,date_of_birth,date_of_proceeding,date_of_publication,federal_state,file_name,kind,title,type_of_proceeding,zipcode,detail_form_name,format,name,register,request_fingerprint,description_hash
1225781,NaN,26 IN 292/21,Tübingen,1988-10-10,2021-09-14,2021-09-14,Baden-Württemberg,NaN,in,NaN,Eröffnungen,75397,tbl_ergebnis:12:frm_detail,neu,Gottlewski,NaN,d3f7c2a8c3fe7c3603a055a99d02b9a70e24d6927c3d5b...,2263c31209d9600992e08b63d8d3e461046e163e6ce7ab...
1702052,NaN,IN 204/21,Hof,1957-07-05,2021-09-10,2021-09-13,Bayern,NaN,in,NaN,Eröffnungen,95233,tbl_ergebnis:4:frm_detail,neu,"Richter, Angelika Sabine",NaN,6e434d745f195df6e9d5486fc42872130fcf8cec34b74f...,2de256a6842e6ef2717fcef67d2a97aacf6e707875212a...
1695795,NaN,46 IN 34/21,Mönchengladbach,1973-02-09,2021-08-31,2021-08-31,Nordrhein-Westfalen,NaN,in,NaN,Eröffnungen,41236,tbl_ergebnis:19:frm_detail,neu,"Sommer, Christoph",NaN,3800f6815a3beb24457713ecb76f3610b90d97834ea00c...,09e6426a65e3050c9ab6f930a5abcf3c23498811ab2a70...
1710062,NaN,903 IN 376/21 - 0 -,Hannover,1970-06-16,NaT,2021-09-29,Niedersachsen,NaN,in,NaN,Eröffnungen,30625,tbl_ergebnis:1:frm_detail,neu,"Feise-Klöfkorn, Frauke",NaN,2c6d5fe4d1f18165f7f8768ea8de367b0e878d6e47011b...,723dce0308f7a481b98153ad67e8eb843c520b322f4470...
1749393,NaN,IN 315/21,Würzburg,1988-10-12,2021-10-11,2021-10-12,Bayern,NaN,in,NaN,Eröffnungen,97082,tbl_ergebnis:24:frm_detail,neu,"Kalencuk , Sergej Viktorovic",NaN,e067aafe05e506975da0b222d002647b73c2097e30594b...,cd4be14cd37228d1cb448c2a8509833c70359f8536b3cf...
1582439,NaN,58 IN 548/21,Freiburg im Breisgau,1946-08-25,2021-10-04,2021-10-06,Baden-Württemberg,NaN,in,NaN,Eröffnungen,79114,tbl_ergebnis:22:frm_detail,neu,"Kenk, Hartmut Walter",NaN,24f6fdea8bed498a2978283de704c7664fb8ebba51edaa...,b95b14e50e75d1282366af29c518e752b71f4f27c6d3e9...
1686836,NaN,610 IN 201/21,Rosenheim,1964-11-07,2021-08-09,2021-08-10,Bayern,NaN,in,NaN,Eröffnungen,83043,tbl_ergebnis:5:frm_detail,neu,"di Benedetto, Giovanni",NaN,5275a232fd6a8c15ffc6e6d01cdbfb5aee092019829411...,831425ac54994744489d882c05a44fcff4ccc8e83170be...
1686825,NaN,1500 IN 1966/21,München,1961-12-24,NaT,2021-08-10,Bayern,NaN,in,NaN,Eröffnungen,80339,tbl_ergebnis:16:frm_detail,neu,"Schwing, Wolfgang",NaN,6b5b772df8c91e9efa9e798cf1dbecc0000ba8cfe82723...,f23552b3004df08298cd9918e3a97894278adc232945a3...
1694018,NaN,70c IN 219/20,Köln,NaT,2021-08-20,2021-08-26,Nordrhein-Westfalen,NaN,in,NaN,Eröffnungen,78999,tbl_ergebnis:12:frm_detail,neu,Handyfloor GmbH,"Köln, HRB 78999",c78bb9d6acdb0557b97cbe410ebac1a607a5ac8d03d914...,ebf653571dd10552775b9f2f90e4b5f6bd863a03e1fc85...
1687792,NaN,93 IN 88/21,Aachen,1983-01-14,2021-08-02,2021-08-12,Nordrhein-Westfalen,NaN,in,NaN,Eröffnungen,53894,tbl_ergebnis:9:frm_detail,neu,"El Mahmoud, Mohamad",NaN,ca3f57a24f7ae40fbeff2ca3fd595aaeb56c7d46250371...,a08b8b202f502d1a69649db431e5bfa19c0ee93646b95a...


#### Group by ZIP and count cases

In [169]:
eröffnung_court_nw = date_range_nw.groupby(['zipcode'], as_index=False).count()[['zipcode','description_hash']]
eröffnung_court_nw.sample(10)

,zipcode,description_hash
1125,33189,2
2894,90431,4
974,29345,1
31,01904,1
1019,30539,3
1891,56743,1
273,10965,3
409,14348,1
1863,56170,1
1609,48167,1


#### Join with OSM ZIP database

In [170]:
df_osm_plz = pd.read_csv("plz_de.csv", dtype=str)
df_osm_plz = df_osm_plz[["plz", "ags", "einwohner", "landkreis", "ort"]]

df_osm_plz.rename(columns={"plz": "zipcode"}, inplace=True)
df_osm_plz = df_osm_plz.groupby("zipcode").last()


eröffnung_nw_osm = eröffnung_court_nw.join(df_osm_plz, on=["zipcode"], how="outer")
eröffnung_nw_osm.reset_index(inplace=True, drop=True)
eröffnung_nw_osm.sample(50)

,zipcode,description_hash,ags,einwohner,landkreis,ort
7806,93199,NaN,09372167,1851,Landkreis Cham,Zell
1247,37619,1.0,03255003,9669,Landkreis Holzminden,Bodenwerder
6244,74582,NaN,08127032,4239,Landkreis Schwäbisch Hall,Gerabronn
6136,73340,NaN,08425008,3891,Alb-Donau-Kreis,Amstetten
293,12047,3.0,11000000,13918,None,Berlin
5415,56594,NaN,07132116,849,Landkreis Altenkirchen,Willroth
3809,21388,NaN,03355034,1489,Landkreis Lüneburg,Soderstorf
2367,73035,3.0,08117026,19575,Landkreis Göppingen,Göppingen
6581,79695,NaN,08336096,568,Landkreis Lörrach,Wieden
4762,37355,NaN,16061074,5544,Landkreis Eichsfeld,Niederorschel


In [171]:
eröffnung_nw_osm[eröffnung_nw_osm["zipcode"] == "33154"]

,zipcode,description_hash,ags,einwohner,landkreis,ort
1120,33154,1.0,05774032,24511,Kreis Paderborn,Paderborn


In [172]:
eröffnung_nw_osm.einwohner = eröffnung_nw_osm.einwohner.astype(float)

# Fülle leere landkreis zellen mit dem entsprechenden wert aus ort
eröffnung_nw_osm.loc[eröffnung_nw_osm["landkreis"].isnull(), ["landkreis"]] = eröffnung_nw_osm["ort"]
eröffnung_nw_osm.sample(10)

,zipcode,description_hash,ags,einwohner,landkreis,ort
6301,75053,NaN,08215025,3365.0,Landkreis Karlsruhe,Gondelsheim
5340,55608,NaN,07133010,1751.0,Landkreis Bad Kreuznach,Becherbach bei Kirn
1972,59505,2.0,05974008,11517.0,Kreis Soest,Bad Sassendorf
7148,86860,NaN,09777140,2400.0,Landkreis Ostallgäu,Jengen
4973,45138,NaN,05113000,17683.0,Essen,Essen
8230,97486,NaN,09674164,3708.0,Landkreis Haßberge,Königsberg in Bayern
7788,93149,NaN,09376149,8386.0,Landkreis Schwandorf,Nittenau
6923,84571,NaN,09171129,2615.0,Landkreis Altötting,Reischach
767,24159,1.0,01002000,19917.0,Kiel,Kiel
6832,84088,NaN,09274153,3815.0,Landkreis Landshut,Neufahrn in Niederbayern


##### ZIP Karte

In [173]:
eröffnung_zip = eröffnung_nw_osm.copy()

In [174]:
eröffnung_zip["ratio"] = eröffnung_zip["description_hash"] / eröffnung_zip["einwohner"] * 1000

In [175]:
eröffnung_zip.to_csv(f'eröffnung_zip_nw_90T_{FILTER_ART}_1000.csv')

##### Gemeinden-Karte

In [176]:
eröffnung_ort = eröffnung_nw_osm.copy()

In [177]:
eröffnung_ort = eröffnung_ort.drop(columns=["zipcode"])

In [178]:
eröffnung_ort = eröffnung_ort.groupby(["ags", "ort"], as_index=False).sum()

In [179]:
eröffnung_ort.head()

,ags,ort,description_hash,einwohner
0,01001000,Flensburg,7.0,82730.0
1,01002000,Kiel,17.0,245155.0
2,01003000,Lübeck,15.0,209583.0
3,01004000,Neumünster,6.0,77651.0
4,01051001,Albersdorf,0.0,6198.0


In [180]:
eröffnung_ort.isnull().sum()

ags                 0
ort                 0
description_hash    0
einwohner           0
dtype: int64

In [181]:
eröffnung_ort["ratio"] = eröffnung_ort["description_hash"] / eröffnung_ort["einwohner"] * 100000

In [182]:
eröffnung_ort.to_csv(f'90T_joined_{FILTER_ART}_eröffnet_bund_gemeinde_100000.csv')

##### Kreis-Karte

In [183]:
eröffnung_kreis = eröffnung_nw_osm.copy()

In [184]:
eröffnung_kreis = eröffnung_kreis.drop(columns=["zipcode", "ort"])

eröffnung_kreis.ags = eröffnung_kreis.ags.str.slice(0, 5)

In [185]:
eröffnung_kreis = eröffnung_kreis.groupby(["ags", "landkreis"], as_index=False).sum()

In [186]:
eröffnung_kreis.head()

,ags,landkreis,description_hash,einwohner
0,01001,Flensburg,7.0,82730.0
1,01002,Kiel,17.0,245155.0
2,01003,Lübeck,15.0,209583.0
3,01004,Neumünster,6.0,77651.0
4,01051,Kreis Dithmarschen,17.0,133223.0


In [187]:
eröffnung_kreis.isnull().sum()

ags                 0
landkreis           0
description_hash    0
einwohner           0
dtype: int64

In [188]:
eröffnung_kreis["ratio"] = eröffnung_kreis["description_hash"] / eröffnung_kreis["einwohner"] * 100000

In [189]:
eröffnung_kreis.to_csv(f'90T_joined_{FILTER_ART}_eröffnet_bund_kreis_100000.csv')

In [140]:
len(eröffnung_kreis.landkreis.unique())

399

In [141]:
eröffnung_kreis.ags.unique()

array(['01001', '01002', '01003', '01004', '01051', '01053', '01054',
       '01055', '01056', '01057', '01058', '01059', '01060', '01061',
       '01062', '02000', '03101', '03102', '03103', '03151', '03153',
       '03154', '03155', '03157', '03158', '03159', '03241', '03251',
       '03252', '03254', '03255', '03256', '03257', '03351', '03352',
       '03353', '03354', '03355', '03356', '03357', '03358', '03359',
       '03360', '03361', '03401', '03402', '03403', '03404', '03405',
       '03451', '03452', '03453', '03454', '03455', '03456', '03457',
       '03458', '03459', '03460', '03461', '03462', '04011', '04012',
       '05111', '05112', '05113', '05114', '05116', '05117', '05119',
       '05120', '05122', '05124', '05154', '05158', '05162', '05166',
       '05170', '05314', '05315', '05316', '05334', '05358', '05362',
       '05366', '05370', '05374', '05378', '05382', '05512', '05513',
       '05515', '05554', '05558', '05562', '05566', '05570', '05711',
       '05754', '057

In [30]:
nw_eröffnet_ik_week = (
    data_nw_ik[["date_of_publication", "description_hash"]]
    .groupby([pd.Grouper(key="date_of_publication", freq="W-MON", label="left")])
    .count()
)

In [44]:
df.federal_state.unique()

array(['bb', 'nw', 'sh', 'ns', 'by', 'sn', 'bw', 'be', 'st', 'he', 'rp',
       'hb', 'mv', 'hh', 'sl', 'th', 'Bayern', 'Rheinland-Pfalz',
       'Thüringen', 'Schleswig-Holstein', 'Sachsen-Anhalt', 'Sachsen',
       'Saarland', 'Nordrhein-Westfalen', 'Niedersachsen',
       'Mecklenburg-Vorpommern', 'Hessen', 'Hamburg', 'Bremen',
       'Brandenburg', 'Berlin', 'Baden-Württemberg'], dtype=object)

In [ ]:
# IK = Verbraucherinsolvenz

def ik_kind(df):
    ik_verbraucher = (df[df['kind'] == 'ik'])
    return ik_verbraucher

# Funktion def wird ausgeführt
data_ik = ik_kind(df)

### Art des Verfahrens

In [18]:
df.type_of_proceeding.unique()

array(['Entscheidungen_im_Verfahren', 'Eroeffnung',
       'Entscheidungen_im_Restschuldbefreiungsverfahren', 'Termine',
       'Entscheidungen_nach_Aufhebung',
       'Verwalter_Treuhaender_Verteilungsverzeichnis',
       'Entscheidungen_im_Verfahren_Aufhebung_Einstellung',
       'Restschuldbefreiung', 'Entscheidungen_im_Verfahren_mit_Termine',
       'Sonstiges', 'Bestimmung_Termine', 'InsO_d_Verw_Treuh_',
       'Sonstiges_ausserhalb_des_Verfahresn', 'Eroeffnungen',
       'Eroeffnung_Insolvenzverfahren',
       'Restschuldbefreiung_Erteilung_Versagung',
       'Abweisungen_mangels_Masse', 'Sicherungsmassnahmen',
       'Anordnung_Sicherungsmassnahmen',
       'Verteilungsverzeichnis_Nachtragsvert_Loeschung',
       'Entscheidungen_nach_Aufhebung_des_Verfahrens',
       'Sicherungsmassnahme_Aufhebung', 'InsO_Plan_Ueberwachung',
       'InsO_Plan_Aufhebung_Ueberwachung', 'Ueberwachte_Insolvenzplaene',
       'InsO_Plan_Ankuendigung_Ueberwachung',
       'Verteilungsverzeichnisse_(§_

In [25]:
def in_eröffnung(data_in):
    in_eröffnet = data_in[data_in['type_of_proceeding'].isin(['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren', 'Eroeffnungen'])]
    return in_eröffnet

# Funktion def wird ausgeführt
data_eröffnet_in = in_eröffnung(data_in)

In [36]:
data_eröffnet_in.to_csv('data_eröffnung_in_bund.csv')

# Verbraucher

In [ ]:
def ik_eröffnung(df):
    ik_eröffnet = df[df['type_of_proceeding'].isin(['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren', 'Eroeffnungen'])]
    return ik_eröffnet

# Funktion def wird ausgeführt
data_eröffnet = ik_eröffnung(df)

In [ ]:
def ik_masse(df):
    ik_mangels_masse = df[df['type_of_proceeding'].isin(['Abweisungen_mangels_Masse'])]
    return ik_mangels_masse

# Funktion def wird ausgeführt
data_masse = ik_masse(df)

In [ ]:
def ik_sicher(df):
    ik_sicherung = df[df['type_of_proceeding'].isin(['Anordnung_Sicherungsmassnahmen', 'Sicherungsmassnahmen'])]
    return ik_sicherung

# Funktion def wird ausgeführt
data_sicher = ik_sicher(df)

### groupby Bundesland descrptionhash

### groupby zipcode Bundesweit

In [ ]:
freq_in_nw_eröffnung_zipcode = data_nw_eröffnet.groupby(['zipcode'], as_index=False).count()
freq_in_nw_eröffnung_zipcode.sample(10)

### Groupby Inolvenzgericht

### Groupby week

In [32]:
data_eröffnet_in_week = (
    data_eröffnet_in[["date_of_publication", "description_hash"]]
    .groupby(pd.Grouper(key="date_of_publication", freq="W-MON", label="left"))
    .count()
)

In [33]:
data_eröffnet_in_week.head()

,description_hash
date_of_publication,
2018-12-31,392
2019-01-07,249
2019-01-14,231
2019-01-21,233
2019-01-28,445


In [35]:
data_eröffnet_in_week.to_csv('freq_in_eröffnung_bund.csv')

### Alter 

In [ ]:
freq_in_nw_eröffnung_birth = freq_in_nw_eröffnung_birth.groupby(['date_of_birth'], as_index=False).count()
freq_in_nw_eröffnung_birth.sample(10)

In [6]:
data_nw = df[df["federal_state"].isin(['Nordrhein-Westfalen', "nw"])]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204015 entries, 1341 to 1819821
Data columns (total 18 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   _key                 8959 non-null    object        
 1   case_nr              204015 non-null  object        
 2   court                204015 non-null  object        
 3   date_of_birth        103479 non-null  datetime64[ns]
 4   date_of_proceeding   203559 non-null  datetime64[ns]
 5   date_of_publication  204015 non-null  datetime64[ns]
 6   description_hash     204015 non-null  object        
 7   federal_state        204015 non-null  object        
 8   file_name            101866 non-null  object        
 9   kind                 203915 non-null  object        
 10  title                101866 non-null  object        
 11  type_of_proceeding   204015 non-null  object        
 12  zipcode              203738 non-null  object        
 13  detail_for

In [7]:
data_nw.isnull().sum() # die Hälfte der Einträge hat keinen Wert in den geburtsdaten. Es macht also keinen Sinn hier eine Berechnung anzuwenden, da die Fehlerquote ienfach hoch wäre.

_key                   195056
case_nr                     0
court                       0
date_of_birth          100536
date_of_proceeding        456
date_of_publication         0
description_hash            0
federal_state               0
file_name              102149
kind                      100
title                  102149
type_of_proceeding          0
zipcode                   277
detail_form_name       101866
format                 101866
name                   101866
register               187410
request_fingerprint    101866
dtype: int64

In [9]:
def nw_eröffnung(data_nw):
    nw_eröffnet = data_nw[data_nw['type_of_proceeding'].isin(['Eröffnungen', 'Eroeffnung', 'Eroeffnung_Insolvenzverfahren'])]
    return nw_eröffnet

# Funktion def wird ausgeführt
data_nw_eröffnet = nw_eröffnung(data_nw)

In [11]:
data_nw_eröffnet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32800 entries, 21335 to 1818646
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   _key                 507 non-null    object        
 1   case_nr              32800 non-null  object        
 2   court                32800 non-null  object        
 3   date_of_birth        26980 non-null  datetime64[ns]
 4   date_of_proceeding   32666 non-null  datetime64[ns]
 5   date_of_publication  32800 non-null  datetime64[ns]
 6   description_hash     32800 non-null  object        
 7   federal_state        32800 non-null  object        
 8   file_name            9769 non-null   object        
 9   kind                 32781 non-null  object        
 10  title                9769 non-null   object        
 11  type_of_proceeding   32800 non-null  object        
 12  zipcode              32790 non-null  object        
 13  detail_form_name     2303

In [10]:
data_nw_eröffnet.isnull().sum() # 21,57 % der Werte fehlen !

_key                   32293
case_nr                    0
court                      0
date_of_birth           5820
date_of_proceeding       134
date_of_publication        0
description_hash           0
federal_state              0
file_name              23031
kind                      19
title                  23031
type_of_proceeding         0
zipcode                   10
detail_form_name        9769
format                  9769
name                    9769
register               27911
request_fingerprint     9769
dtype: int64